In [1]:
import time
import pandas as pd
import pyarrow as pa

from shared_code.utility.spark.set_environ import set_azure_env
from shared_code.utility.storage.azure_file_storage import AzureFileStorageAdapter
set_azure_env()

In [2]:
fs_adapter = AzureFileStorageAdapter('data')
file_system = fs_adapter.get_file_storage()

In [35]:
%%time
extant_data = pd.read_parquet("data/processed_raw_data.parquet", engine='pyarrow', filesystem=file_system)
display(extant_data)

,id,subreddit,author,title,caption,hash,permalink,original_url,image_name,path,thumbnail_path,exists,curated
0,1000cej,CityPorn,OtterlyFoxy,New York in the fog,cars are parked on the side of the road in the...,7a8d96e378c15c8ab8440ac311f12c11,/r/CityPorn/comments/1000cej/new_york_in_the_fog/,https://i.redd.it/4emw5uldib9a1.jpg,4emw5uldib9a1.jpg,D:\data\images\CityPorn\4emw5uldib9a1.jpg,D:\data\images\CityPorn\thumbnail\4emw5uldib9a...,True,False
1,1000glf,AmIhotAF,toolate_sharkbait,Just looking for entertainment,blonde woman with blonde hair and tattoos on h...,e554c1ed7ffa2740436ac082068b2824,/r/AmIhotAF/comments/1000glf/just_looking_for_...,https://i.redd.it/4xyb1vgbjb9a1.jpg,4xyb1vgbjb9a1.jpg,D:\data\images\AmIhotAF\4xyb1vgbjb9a1.jpg,D:\data\images\AmIhotAF\thumbnail\4xyb1vgbjb9a...,True,False
2,1000j1n,greentext,trent8051,Anon wants Elon cut,a man with a beard and a beard sitting in fron...,1dec3dabb5e46cde01855d06089c287a,/r/greentext/comments/1000j1n/anon_wants_elon_...,https://i.redd.it/3mewbe0wjb9a1.jpg,3mewbe0wjb9a1.jpg,D:\data\images\greentext\3mewbe0wjb9a1.jpg,D:\data\images\greentext\thumbnail\3mewbe0wjb9...,True,False
3,1000mjs,spaceporn,MorningStar_imangi,Northern Lights above Lofoten,a view of a view of a large green and purple a...,2c39ce1290fba541abd0b004b09da6b2,/r/spaceporn/comments/1000mjs/northern_lights_...,https://i.redd.it/7s5aafaqkb9a1.jpg,7s5aafaqkb9a1.jpg,/data/images/spaceporn/7s5aafaqkb9a1.jpg,/data/images/spaceporn/thumbnail/7s5aafaqkb9a1...,True,False
4,1000qpd,spaceporn,MorningStar_imangi,Viking Lights,a scene of a boat is sitting on the shore of a...,0f72de47c69ff50eca5fa3990215f4ac,/r/spaceporn/comments/1000qpd/viking_lights/,https://i.redd.it/abojw7lqlb9a1.jpg,abojw7lqlb9a1.jpg,/data/images/spaceporn/abojw7lqlb9a1.jpg,/data/images/spaceporn/thumbnail/abojw7lqlb9a1...,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
22482,117wxlw,celebrities,Hackshell1,Crush Alexandra Daddario,a woman in a green dress is posing on a couch,92df1c77858e51f6b192de1b41b958eb,/r/celebrities/comments/117wxlw/crush_alexandr...,https://i.redd.it/azzjcehs5jja1.jpg,92df1c77858e51f6b192de1b41b958eb.jpg,D:\data\images\celebrities\92df1c77858e51f6b19...,D:\data\images\celebrities\thumbnail\92df1c778...,True,False
22483,117nncw,celebrities,Moviebuff1978,Kira Kosarin,woman in a bikinisuit with a cell phone phone ...,29b45617722adb1ebcb38415b5045f75,/r/celebrities/comments/117nncw/kira_kosarin/,https://i.redd.it/wwnpdsslvgja1.jpg,29b45617722adb1ebcb38415b5045f75.jpg,D:\data\images\celebrities\29b45617722adb1ebcb...,D:\data\images\celebrities\thumbnail\29b456177...,True,False
22484,1170f84,celebrities,iamtheworstgood,Katherine Langford,a woman in a black dress with a white dress an...,ecf5302efee275f4ab73b078e5b5ea26,/r/celebrities/comments/1170f84/katherine_lang...,https://i.redd.it/k024488x4cja1.jpg,ecf5302efee275f4ab73b078e5b5ea26.jpg,D:\data\images\celebrities\ecf5302efee275f4ab7...,D:\data\images\celebrities\thumbnail\ecf5302ef...,True,False
22485,1169tnd,celebrities,ashishkhakural5,Kate winslet,a close up of a woman with a necklace and a ne...,d943d2f923551046606f15afab82b3fa,/r/celebrities/comments/1169tnd/kate_winslet/,https://i.redd.it/zp072p0nq6ja1.jpg,d943d2f923551046606f15afab82b3fa.jpg,D:\data\images\celebrities\d943d2f923551046606...,D:\data\images\celebrities\thumbnail\d943d2f92...,True,False


CPU times: total: 188 ms
Wall time: 777 ms


In [36]:
%%time

columns = ['id', 'subreddit', 'title', 'caption', 'image_name', 'exists', 'curated']

slim_df = extant_data[columns]

exists = slim_df\
    .where(slim_df['exists'] == True)\
    .dropna(axis=1)\
    .dropna(how='all')\
    .reset_index(drop=True)

display(exists)

,id,subreddit,title,caption,image_name,exists,curated
0,1000cej,CityPorn,New York in the fog,cars are parked on the side of the road in the...,4emw5uldib9a1.jpg,True,False
1,1000glf,AmIhotAF,Just looking for entertainment,blonde woman with blonde hair and tattoos on h...,4xyb1vgbjb9a1.jpg,True,False
2,1000j1n,greentext,Anon wants Elon cut,a man with a beard and a beard sitting in fron...,3mewbe0wjb9a1.jpg,True,False
3,1000mjs,spaceporn,Northern Lights above Lofoten,a view of a view of a large green and purple a...,7s5aafaqkb9a1.jpg,True,False
4,1000qpd,spaceporn,Viking Lights,a scene of a boat is sitting on the shore of a...,abojw7lqlb9a1.jpg,True,False
...,...,...,...,...,...,...,...
22482,117wxlw,celebrities,Crush Alexandra Daddario,a woman in a green dress is posing on a couch,92df1c77858e51f6b192de1b41b958eb.jpg,True,False
22483,117nncw,celebrities,Kira Kosarin,woman in a bikinisuit with a cell phone phone ...,29b45617722adb1ebcb38415b5045f75.jpg,True,False
22484,1170f84,celebrities,Katherine Langford,a woman in a black dress with a white dress an...,ecf5302efee275f4ab73b078e5b5ea26.jpg,True,False
22485,1169tnd,celebrities,Kate winslet,a close up of a woman with a necklace and a ne...,d943d2f923551046606f15afab82b3fa.jpg,True,False


CPU times: total: 46.9 ms
Wall time: 41.1 ms


In [43]:
%%time

foo = exists.rename(columns={'id':'Id', 'subreddit': 'SubReddit', 'title':"Title", 'caption': 'Caption', 'image_name': 'ImageName', 'exists': 'Exists', 'curated': 'Curated'})

display(foo)

,Id,SubReddit,Title,Caption,ImageName,Exists,Curated
0,1000cej,CityPorn,New York in the fog,cars are parked on the side of the road in the...,4emw5uldib9a1.jpg,True,False
1,1000glf,AmIhotAF,Just looking for entertainment,blonde woman with blonde hair and tattoos on h...,4xyb1vgbjb9a1.jpg,True,False
2,1000j1n,greentext,Anon wants Elon cut,a man with a beard and a beard sitting in fron...,3mewbe0wjb9a1.jpg,True,False
3,1000mjs,spaceporn,Northern Lights above Lofoten,a view of a view of a large green and purple a...,7s5aafaqkb9a1.jpg,True,False
4,1000qpd,spaceporn,Viking Lights,a scene of a boat is sitting on the shore of a...,abojw7lqlb9a1.jpg,True,False
...,...,...,...,...,...,...,...
22482,117wxlw,celebrities,Crush Alexandra Daddario,a woman in a green dress is posing on a couch,92df1c77858e51f6b192de1b41b958eb.jpg,True,False
22483,117nncw,celebrities,Kira Kosarin,woman in a bikinisuit with a cell phone phone ...,29b45617722adb1ebcb38415b5045f75.jpg,True,False
22484,1170f84,celebrities,Katherine Langford,a woman in a black dress with a white dress an...,ecf5302efee275f4ab73b078e5b5ea26.jpg,True,False
22485,1169tnd,celebrities,Kate winslet,a close up of a woman with a necklace and a ne...,d943d2f923551046606f15afab82b3fa.jpg,True,False


CPU times: total: 0 ns
Wall time: 0 ns


In [46]:
%%time

from shared_code.utility.storage.table import TableAdapter

tableAdapter: TableAdapter = TableAdapter()

curation_table = tableAdapter.get_table_client('curation')

records = foo.to_dict(orient='records')

for record in records:
    record['PartitionKey'] = record['SubReddit']
    record['RowKey'] = record['Id']
    curation_table.create_entity(entity=record)

KeyboardInterrupt: 